In [140]:
import pandas as pd

passed_data = []
data_procession = []
with open('/home/ouconstand/data/Virus_Host/Virion.csv','r', encoding='ISO-8859-1') as f:
    for line in f.read().splitlines():
        data = line.split('\t')
        if len(data) < 34:
            passed_data.append(data)
        else:
            if data[8] == '' or data[13] == '': # delete non_family information data
                pass
            else:
                data_procession.append(data)

data = pd.DataFrame(data_procession[1:-1],columns=data_procession[0])

In [141]:
data.shape

(480191, 34)

In [142]:
len(passed_data)

12

In [27]:
print(passed_data[0])
print(passed_data[3])
print(passed_data[6])
print(passed_data[9])

['capra hircus', 'small ruminant morbillivirus', '9925', '31604', 'TRUE', 'TRUE', 'TRUE', 'capra', 'bovidae', 'artiodactyla', 'mammalia', 'capra hircus', 'morbillivirus', 'paramyxoviridae', 'mononegavirales', 'monjiviricetes', 'peste-des-petits-ruminants virus', 'FALSE', 'Isolation/Observation', 'Isolation', 'Shaw', 'Shaw et al. 2020 Mol Ecol', '', '"ViPR']
['capra ibex', 'small ruminant morbillivirus', '72542', '31604', 'TRUE', 'TRUE', 'TRUE', 'capra', 'bovidae', 'artiodactyla', 'mammalia', 'capra ibex', 'morbillivirus', 'paramyxoviridae', 'mononegavirales', 'monjiviricetes', 'peste-des-petits-ruminants virus', 'FALSE', 'PCR/Sequencing', 'PCR', 'Shaw', 'Shaw et al. 2020 Mol Ecol', '', '"ViPR']
['gazella dorcas', 'small ruminant morbillivirus', '37751', '31604', 'TRUE', 'TRUE', 'TRUE', 'gazella', 'bovidae', 'artiodactyla', 'mammalia', 'gazella dorcas', 'morbillivirus', 'paramyxoviridae', 'mononegavirales', 'monjiviricetes', 'peste-des-petits-ruminants virus', 'FALSE', 'PCR/Sequencing',

In [77]:
import pandas as pd
train_data = data[data['VirusFamily'] == 'coronaviridae']
virusID_hostfamily = train_data[['VirusTaxID','HostFamily']]
print(virusID_hostfamily.shape)
virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True)
virusID_hostfamily.reset_index(drop=True)
print(virusID_hostfamily.shape)

(40168, 2)
(1338, 2)


<ipython-input-77-5a26d1d3bdf9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True)


In [143]:
import pandas as pd

final_data = pd.DataFrame(columns=['Virus_Family', 'Type', 'Value'])

virus_family = set(data['VirusFamily'].to_list())

n = 0
for one_family in virus_family:
    
    train_data = data[data['VirusFamily'] == one_family]
    virusID_hostfamily = train_data[['VirusTaxID','HostFamily']]
    virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
    virusID_hostfamily.reset_index(drop=True) # reset index

    dic = {}
    total = 0
    homo_trans = 0
    homo_non_trans = 0
    non_homo_trans = 0
    non_homo_non_trans = 0

    for i,j in zip(virusID_hostfamily['VirusTaxID'], virusID_hostfamily['HostFamily']):
        dic[i] = dic.get(i, []) + [j]
    for value in dic.values():
        if 'hominidae' in value:
            if len(value) > 1:
                homo_trans += 1
            else:
                homo_non_trans += 1
        else:
            if len(value) > 1:
                non_homo_trans += 1
            else:
                non_homo_non_trans += 1
        total += 1
        
    if total != 0:
        final_data.loc[(4*n)+0] = [one_family, 'homo_trans', round(homo_trans/total, 5)]
        final_data.loc[(4*n)+1] = [one_family, 'homo_non_trans', round(homo_non_trans/total, 5)]
        final_data.loc[(4*n)+2] = [one_family, 'non_homo_trans', round(non_homo_trans/total, 5)]
        final_data.loc[(4*n)+3] = [one_family, 'non_homo_non_trans', round(non_homo_non_trans/total, 5)]
    n += 1

<ipython-input-143-f4abfdf098a1>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
<ipython-input-143-f4abfdf098a1>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
<ipython-input-143-f4abfdf098a1>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [144]:
final_data.to_csv('/mnt/c/Users/ouyangkang/Desktop/final_data.csv',index=None)